<a href="https://colab.research.google.com/github/OFFplanet/English-Hindi-Transliteration-/blob/main/Devnagri_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/HE_Trans

Mounted at /content/drive
/content/drive/MyDrive/HE_Trans


In [2]:
pip install tensorflow-addons


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.3 MB/s eta 0:00:00


In [3]:
import nltk
from collections import Counter
from tqdm import tqdm_notebook
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_addons.seq2seq as seq2seq
from tensorflow.compat.v1.nn.rnn_cell import DropoutWrapper
from tensorflow.compat.v1 import get_variable, placeholder
from tensorflow.compat.v1 import Session as sess
tf.compat.v1.disable_eager_execution()
from tensorflow.nn import embedding_lookup
import random

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
MAX_SEQ_LEN = 20
BATCH_SIZE = 64

In [6]:
class Lang:
    def __init__(self, counter, vocab_size):
        self.word2id = {}
        self.id2word = {}
        self.pad = "<PAD>"
        self.sos = "<SOS>"
        self.eos = "<EOS>"
        self.unk = "<UNK>"
        
        self.ipad = 0
        self.isos = 1
        self.ieos = 2
        self.iunk = 3
        
        self.word2id[self.pad] = 0
        self.word2id[self.sos] = 1
        self.word2id[self.eos] = 2
        self.word2id[self.unk] = 3
        
        self.id2word[0] = self.pad
        self.id2word[1] = self.sos
        self.id2word[2] = self.eos
        self.id2word[3] = self.unk
        
        curr_id = 4
        for w, c in counter.most_common(vocab_size):
            self.word2id[w] = curr_id
            self.id2word[curr_id] = w
            curr_id += 1
            
    def encodeSentence(self, s, max_len=-1):
        wseq = s.lower().strip()
        if max_len == -1:
            return [self.word2id[w] if w in self.word2id else self.iunk for w in wseq]
        else:
            return ([self.word2id[w] if w in self.word2id else self.iunk for w in wseq] + [self.ieos] + [self.ipad]*max_len)[:max_len]
        
    def encodeSentence2(self, s, max_len=-1):
        wseq = wseq = s.lower().strip()
        return min(max_len, len(wseq)+1), \
            ([self.word2id[w] if w in self.word2id else self.iunk for w in wseq] + \
                [self.ieos] + [self.ipad]*max_len)[:max_len]
    
    def decodeSentence(self, id_seq):
        id_seq = np.array(id_seq + [self.ieos])
        j = np.argmax(id_seq==self.ieos)
        s = ''.join([self.id2word[x] for x in id_seq[:j]])
        s = s.replace(self.unk, "UNK")
        return s

In [7]:
# N = 30823
N = 12938

In [8]:
!ls

txtdata1.txt  txtdata2.txt


In [10]:
hi_counter = Counter()
hi_sentences=[]
en_counter = Counter()
en_sentences=[]
with open("txtdata1.txt") as f:
    # for line in tqdm_notebook(f, total=N, desc="Reading file:"):
    for line in f:
        lst = line.strip().split(" ")
        if(len(lst)==2):
          en = lst[0]
          hi = lst [1]
        else:
          en = " ".join(lst[:int(len(lst)/2)])
          hi =  " ".join(lst[int(len(lst)/2):])

        hi_sentences.append(hi)
        en_sentences.append(en)
    for line in tqdm_notebook(hi_sentences, desc="Processing inputs:"):
        for w in line.strip():
            hi_counter[w] += 1
    for line in tqdm_notebook(en_sentences, desc="Processing inputs:"):
        for w in line.strip():
            en_counter[w] += 1

<ipython-input-10-4a0026528454>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(hi_sentences, desc="Processing inputs:"):


Processing inputs::   0%|          | 0/12937 [00:00<?, ?it/s]

<ipython-input-10-4a0026528454>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(en_sentences, desc="Processing inputs:"):


Processing inputs::   0%|          | 0/12937 [00:00<?, ?it/s]

In [11]:
# # A few sample hindi characters
# print("Most common hi characters in dataset:\n", hi_counter.most_common(5))

# print("\nTotal (hi)characters gathered from dataset:",len(hi_counter))

# # A few sample english characters
# print("\nMost common en characters in dataset:\n", en_counter.most_common(5))

# print("\nTotal (en)characters gathered from dataset:", len(en_counter))

Most common hi characters in dataset:
 [('ा', 8875), ('र', 8818), ('्', 7558), (' ', 6200), ('न', 5431)]

Total (hi)characters gathered from dataset: 107

Most common en characters in dataset:
 [('a', 21693), ('r', 9114), ('i', 8660), ('n', 8314), ('e', 8303)]

Total (en)characters gathered from dataset: 48


In [12]:
en_lang = Lang(en_counter, len(en_counter))
hi_lang = Lang(hi_counter, len(hi_counter))

In [14]:
VE = len(en_lang.word2id)
VH = len(hi_lang.word2id)

# print(len(en_counter),len(hi_counter))
# print(VE,VH)

48 107
52 111


In [15]:
en_word_emb_matrix = get_variable("en_word_emb_matrix", (VE, 300), dtype=tf.float32)
hi_word_emb_matrix = get_variable("hi_word_emb_matrix", (VH, 300), dtype=tf.float32)

In [16]:
tf.compat.v1.disable_eager_execution()

In [17]:
keep_prob = placeholder(tf.float32)

input_ids = placeholder(tf.int32, (None, MAX_SEQ_LEN))
input_lens = placeholder(tf.int32, (None, ))

ph_target_ids = placeholder(tf.int32, (None, MAX_SEQ_LEN))
target_lens = placeholder(tf.int32, (None, ))

In [18]:
target_ids = tf.concat([tf.fill([BATCH_SIZE,1], hi_lang.isos), ph_target_ids], -1)

In [19]:
input_emb = embedding_lookup(en_word_emb_matrix, input_ids)
target_emb = embedding_lookup(hi_word_emb_matrix, target_ids[:, :-1])

In [21]:
encoder_cell = tf.compat.v1.nn.rnn_cell.GRUCell(128) # 128 is the dimension of hidden state
encoder_cell = DropoutWrapper(encoder_cell, output_keep_prob=keep_prob) # Adding Dropout for regularization

<ipython-input-21-5d9572f1b8a7>:1: UserWarning: `tf.nn.rnn_cell.GRUCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.GRUCell`, and will be replaced by that in Tensorflow 2.0.
  encoder_cell = tf.compat.v1.nn.rnn_cell.GRUCell(128) # 128 is the dimension of hidden state


In [22]:
enc_outputs, enc_state = tf.compat.v1.nn.dynamic_rnn(
    encoder_cell, # The encoder GRU cell
    input_emb, # Embedded input sequence
    sequence_length=input_lens, 
    initial_state=encoder_cell.zero_state(BATCH_SIZE, dtype=tf.float32)
)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [25]:
decoder_cell = tf.compat.v1.nn.rnn_cell.GRUCell(128)
decoder_cell = DropoutWrapper(decoder_cell, output_keep_prob=keep_prob)

<ipython-input-25-da533120c84a>:1: UserWarning: `tf.nn.rnn_cell.GRUCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.GRUCell`, and will be replaced by that in Tensorflow 2.0.
  decoder_cell = tf.compat.v1.nn.rnn_cell.GRUCell(128)


In [26]:
output_projection = tf.compat.v1.layers.Dense(len(hi_lang.word2id))

In [27]:
# !pip3 install tensorflow==1.13.2

In [28]:
import tensorflow_addons.seq2seq as seq2seq

encoder = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder(
    input_emb, training=True
)
encoder_state = (state_h, state_c)

# Decoder RNN cell with attention
attention_mechanism = tfa.seq2seq.LuongAttention(128, enc_outputs)
decoder_cell = tf.keras.layers.LSTMCell(128)
decoder_cell = tfa.seq2seq.AttentionWrapper(
    decoder_cell,
    attention_mechanism,
    attention_layer_size=128,
    initial_cell_state=encoder_state,
)

# Sampler
sampler = tfa.seq2seq.sampler.TrainingSampler()

# Decoder
projection_layer = tf.keras.layers.Dense(128)
decoder = tfa.seq2seq.BasicDecoder(decoder_cell, sampler, output_layer=projection_layer)




In [29]:
decoder_initial_state = decoder_cell.get_initial_state(inputs=target_emb, dtype=tf.float32)
outputs, a, b = decoder(
    target_emb,
    initial_state=decoder_initial_state,
    sequence_length=target_lens,
    training=True,
)
logits = outputs.rnn_output

In [30]:
output_max_len = tf.reduce_max(b)

In [32]:
infer_helper = seq2seq.GreedyEmbeddingSampler()
infer_helper.initialize(hi_word_emb_matrix, tf.fill([BATCH_SIZE, ], hi_lang.isos), hi_lang.ieos)
infer_decoder = seq2seq.BasicDecoder(decoder_cell, infer_helper, output_layer=projection_layer)

infer_output = tfa.seq2seq.dynamic_decode(decoder = infer_decoder,
                            maximum_iterations = MAX_SEQ_LEN,
                            swap_memory = True,
                            decoder_init_input= hi_word_emb_matrix,
                            decoder_init_kwargs= {
                                'initial_state' : decoder_initial_state,
                                'start_tokens': tf.fill([BATCH_SIZE, ], hi_lang.isos), 'end_token': hi_lang.ieos
                            })

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'decoder/assert_greater/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/check_ops.py", line 1045, in assert_greater
    return _binary_assert('>', 'assert_greater', math_ops.greater, np.greater, x,  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/check_ops.py", line 427, in _binary_assert
    return control_flow_ops.Assert(condition, data, summarize=summarize)  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py", line 1082, in op_dispatch_handler
    return dispatch_target(*args, **kwargs)  File "/usr/local/

In [33]:
# Sequence mask:
# To make sure we don't back-propagate error from output of length positions
masks = tf.sequence_mask(target_lens, output_max_len, dtype=tf.float32, name='masks')

# Loss function - weighted softmax cross entropy
cost = seq2seq.sequence_loss(
    outputs[0],
    target_ids[:, 1:(output_max_len + 1)],
    masks)

# Optimizer
optimizer = tf.compat.v1.train.AdamOptimizer(0.0001)

In [34]:
train_op = optimizer.minimize(cost)

In [35]:
init = tf.compat.v1.global_variables_initializer()

In [36]:
sess_config = tf.compat.v1.ConfigProto()
sess_config.gpu_options.allow_growth = True
sess = tf.compat.v1.InteractiveSession(config=sess_config)
sess.run(init)

In [52]:
random.seed(41)
parallel = list(zip(en_sentences, hi_sentences))
random.shuffle(parallel)
parallel[1000]
('hazaarii', 'हज़ारी')
train_n = int(0.95*N)
valid_n = N - train_n
train_pairs = parallel[:train_n].copy()
valid_pairs = parallel[train_n:]
def small_test():
    all_bleu = []
    smoothing = nltk.translate.bleu_score.SmoothingFunction().method7
    for m in range(0, valid_n, BATCH_SIZE):
        # print(f"Status: {m}/{N}", end='\r')
        n = m + BATCH_SIZE
        if n > valid_n:
            # print("Epoch Complete...")
            break

        input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
        input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
        for i in range(m, n):
            b,a = en_lang.encodeSentence2(valid_pairs[i][0], MAX_SEQ_LEN)
            input_batch[i-m,:] = a
            input_lens_batch[i-m] = b

        feed_dict={
            input_ids: input_batch,
            input_lens: input_lens_batch,
            #target_ids: target_batch,
            #target_lens: target_lens_batch,
            keep_prob: 1.0
        }
        pred_batch = sess.run(infer_output[0].sample_id, feed_dict=feed_dict)
        for k, pred_ in enumerate(pred_batch):
            pred_s = hi_lang.decodeSentence(list(pred_))
            ref = valid_pairs[m+k][1]
            try:
                _bx = nltk.translate.bleu_score.sentence_bleu(
                    [ref],
                    pred_s,
                    weights=[1/4]*4,
                    smoothing_function=smoothing)
            except ZeroDivisionError:
                _bx = 0
            all_bleu.append(_bx)

    print(f"BLEU Score: {np.mean(all_bleu)}")
for _e in range(50):
    # Mix things up a bit.
    random.shuffle(train_pairs)
    pbar = tqdm_notebook(range(0, train_n, BATCH_SIZE))
    batch_loss = 0
    bxi = 0
    for m in pbar:
        n = m + BATCH_SIZE
        if n <= train_n:
            # print("Epoch Complete... \n")

            input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
            input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
            for i in range(m, n):
                b,a = en_lang.encodeSentence2(train_pairs[i][0], MAX_SEQ_LEN)
                input_batch[i-m,:] = a
                input_lens_batch[i-m] = b

            target_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
            target_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
            for i in range(m, n):
                b,a = hi_lang.encodeSentence2(train_pairs[i][1], MAX_SEQ_LEN)
                target_batch[i-m,:] = a
                target_lens_batch[i-m] = b

            feed_dict={
                input_ids: input_batch,
                input_lens: input_lens_batch,
                ph_target_ids: target_batch,
                target_lens: target_lens_batch,
                keep_prob: 0.8 
            }
            sess.run(train_op, feed_dict=feed_dict)
            batch_loss += sess.run(cost, feed_dict=feed_dict)
            pbar.set_description(f"Epoch: {_e} >> Loss: {batch_loss/(bxi+1):2.2F}:")
            bxi += 1
            if (1 + n//BATCH_SIZE) % 100 == 0:
                small_test()


<ipython-input-52-6f2787659b2c>:59: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar = tqdm_notebook(range(0, train_n, BATCH_SIZE))


  0%|          | 0/193 [00:00<?, ?it/s]

BLEU Score: 0.583937659226751


  0%|          | 0/193 [00:00<?, ?it/s]

BLEU Score: 0.5874413293658629


...

BLEU Score: 0.6750433345644125


  0%|          | 0/193 [00:00<?, ?it/s]

BLEU Score: 0.6786461064151834


  0%|          | 0/193 [00:00<?, ?it/s]

BLEU Score: 0.6785104875680766


In [54]:
def transliterate(s):
    input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
    input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
    b,a = en_lang.encodeSentence2(s, MAX_SEQ_LEN)
    input_batch[0, :] = a
    input_lens_batch[0] = b
    
    feed_dict={
        input_ids: input_batch,
        input_lens: input_lens_batch,
        #target_ids: target_batch,
        #target_lens: target_lens_batch,
        keep_prob: 1.0
    }
    pred_batch = sess.run(infer_output[0].sample_id, feed_dict=feed_dict)
    pred_ = pred_batch[0]
    pred_s = hi_lang.decodeSentence(list(pred_))
    # ref = valid_pairs[m+k][1]
    return pred_s

In [55]:
transliterate("khaana")

'खाना'

In [61]:
transliterate("bhai")

'भाई'